In [1]:
import glob
import os
import matplotlib as mlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from numpy import loadtxt
import re
import csv
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.patches as mpatches

In [2]:
#constants


In [3]:
# d0 for lectrons and muons separately\n
def impact_d0(param_point):
    
    folder='hepmc_'+param_point
    outputfolder='hepmc_output_'+param_point
    
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    
    os.makedirs(os.path.dirname(outputfolder+"/ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/d0_e.dat","w")
    open(outputfolder+"/d0_mu.dat","w")
    open(outputfolder+"/pT_e.dat","w")
    open(outputfolder+"/pT_mu.dat","w")
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        d0_l=[]
        d0_e=[]
        d0_mu=[]
        pT_e=[]
        pT_mu=[]
        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 
        
        counte=0
        countmu=0
        countevents=0
        countV0=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding electron/muon related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='11'):
                            counte+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            PT=np.sqrt(P[0]**2+P[1]**2)
                            
                            if ((V[0]!=0)&(V[1]!=0)&(V[2]!=0)): 
                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                d0_e.append(PT*np.sqrt(1-cTheta**2))
                                
                            else: 
                                countV0+=1
                                d0_e.append(0)
                                
                            pT_e.append(PT)    
                            
                            
                        if (hepmc[ip+i][2]=='13'):
                            countmu+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)
                            #print(P)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            PT=np.sqrt(P[0]**2+P[1]**2)
                            
                            if ((V[0]!=0)&(V[1]!=0)&(V[2]!=0)):  
                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                d0_mu.append(PT*np.sqrt(1-cTheta**2))
                                
                            else: 
                                countV0+=1
                                d0_mu.append(0)
                            
                            pT_mu.append(PT)

                        i+=1
        #print(d0_l)
    
        print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
              +"# V(primary) = {}".format(countV0))
        
        #Saving d0 to file
        with open(outputfolder+"/d0_e.dat", 'a+') as foute:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foute, d0_e, fmt='%7f')   
                    
        with open(outputfolder+"/d0_mu.dat", 'a+') as foutmu:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foutmu, d0_mu, fmt='%7f')
                    
        #Saving pT to file
        with open(outputfolder+"/pT_e.dat", 'a+') as foute:
                        np.savetxt(foute, pT_e, fmt='%7f')   
                    
        with open(outputfolder+"/pT_mu.dat", 'a+') as foutmu:
                        np.savetxt(foutmu, pT_mu, fmt='%7f')

In [15]:
# lifetime of chi_+
def lifetime(param_point):
    
    folder='hepmc_'+param_point
    outputfolder='hepmc_output_'+param_point
    
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/"+param_point+"_ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/"+param_point+"_ctau_plus.dat","w")
    open(outputfolder+"/"+param_point+"_DM.dat","w")
    open(outputfolder+"/"+param_point+"_beta_plus.dat","w")
    open(outputfolder+"/"+param_point+"_gamma_plus.dat","w")
    open(outputfolder+"/"+param_point+"_betagamma_plus.dat","w")
    open(outputfolder+"/"+param_point+"_pT_plus.dat","w")
    
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        ctau_plus=[]
        beta_plus=[]
        gamma_plus=[]
        betagamma_plus=[]
        pT_plus=[]
        DM=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 

        countevents=0
        countV0=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            
            #calculating beta and gamma factors of chi_plus
            if (row[2]=='9000007') or (row[2]=='-9000007'):
                
                plus=np.array(row[3:7], float)
                #print(plus)
                
                beta=np.sqrt(plus[0]**2+plus[1]**2+plus[2]**2)/plus[3]
                beta_plus.append(beta)
                gamma_plus.append(1/np.sqrt(1-beta**2))
                betagamma_plus.append(beta/np.sqrt(1-beta**2))
                pT_plus.append(np.sqrt(plus[0]**2+plus[1]**2))
                
            
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding DM related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='9000005'):
                            DM.append(hepmc[ip+i])
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating lifetime and time of flight in lab frame
                            ctau_plus.append(np.sqrt(V[0]**2+V[1]**2+V[2]**2))
            
                              

                        i+=1
        #print(d0_l)
    
        #print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
        #      +"# V(primary) = {}".format(countV0))
        
        #Saving to files 
        with open(outputfolder+"/"+param_point+"_ctau_plus.dat", 'a+') as fout:
                        np.savetxt(fout, ctau_plus, fmt='%7f') 
                    
        with open(outputfolder+"/"+param_point+"_beta_plus.dat", 'a+') as fout:
                        np.savetxt(fout, beta_plus, fmt='%7f') 
                    
        with open(outputfolder+"/"+param_point+"_gamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, gamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_betagamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, betagamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_pT_plus.dat", 'a+') as fout:
                        np.savetxt(fout, pT_plus, fmt='%7f') 
                
        #print(DM)            
        #with open(outputfolder+"/DM.dat", 'a+') as fout:
        #                fout.write(DM)

In [5]:
# lifetime of chi_+
def lifetimeAngle(folder,outputfolder,tanmin,tanmax):
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/ctau_plus.dat","w")
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        ctau_plus=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 
        
        counte=0
        countmu=0
        countevents=0
        countAngle=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding DM related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='9000005'):
                            countDM+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating ctau in the range of angles:
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            tantheta=VT/np.abs(V[2])
                            
                            if ((tanmin<tantheta)&(tantheta<tanmax)):
                                countAngle+=1
                                ctau_plus.append(np.sqrt(V[0]**2+V[1]**2+V[2]**2))

                              

                        i+=1
        #print(d0_l)
    
        print("# events = {}".format(countevents)+"# in range = {}".format(countAngle))
        
        #Saving ctau to file 
        
        with open(outputfolder+"/ctau_plus.dat", 'a+') as fout:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(fout, ctau_plus, fmt='%7f')   

In [58]:
# lifetime of chi_+
def lifetime_lhe(folder):
    
    outputfolder=folder+'_output'
    
    #reading pythia output
    files = sorted(glob.glob( folder+'/*.lhe'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/blank.dat"), exist_ok=True)
    
    #open lhe file
    for filenum, file in enumerate(files):
        
        param_point=re.search('\/(.+?).lhe', file).group(1)
        
        open(outputfolder+"/"+param_point+"_ctau_plus.dat","w")
        open(outputfolder+"/"+param_point+"_DM.dat","w")
        open(outputfolder+"/"+param_point+"_beta_plus.dat","w")
        open(outputfolder+"/"+param_point+"_gamma_plus.dat","w")
        open(outputfolder+"/"+param_point+"_betagamma_plus.dat","w")
        open(outputfolder+"/"+param_point+"_pT_plus.dat","w")
        
        lhe=[]
        ctau_plus=[]
        beta_plus=[]
        gamma_plus=[]
        betagamma_plus=[]
        pT_plus=[]
        DM=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f)):
                if (len(row)>1):
                    lhe.append(row)

        print(len(lhe)) 

        countevents=0
        countV0=0
        for rownum ,row in enumerate(lhe):
            #print(lhe[rownum][0])
            #finding vertex
            print(row)
            if (row[0]=='<event>'):
                countevents+=1
            
            #calculating beta and gamma factors of chi_plus
            if (row[0]=='9000007') or (row[0]=='-9000007'):
                
                plus=np.array(row[6:10], float)
                print(plus)
                
                beta=np.sqrt(plus[0]**2+plus[1]**2+plus[2]**2)/plus[3]
                beta_plus.append(beta)
                gamma_plus.append(1/np.sqrt(1-beta**2))
                betagamma_plus.append(beta/np.sqrt(1-beta**2))
                pT_plus.append(np.sqrt(plus[0]**2+plus[1]**2))
            
                              
        #print(d0_l)
    
        #print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
        #      +"# V(primary) = {}".format(countV0))
        
        #Saving to files 

                    
        with open(outputfolder+"/"+param_point+"_beta_plus.dat", 'a+') as fout:
                        np.savetxt(fout, beta_plus, fmt='%7f') 
                    
        with open(outputfolder+"/"+param_point+"_gamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, gamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_betagamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, betagamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/"+param_point+"_pT_plus.dat", 'a+') as fout:
                        np.savetxt(fout, pT_plus, fmt='%7f') 
                
        #print(DM)            
        #with open(outputfolder+"/DM.dat", 'a+') as fout:
        #                fout.write(DM)

In [59]:
lifetime_lhe('lhe_files_ctau2cm')

['lhe_files_ctau2cm/rr5.00e-08_mS2.75e+02_mT3.24e+02.lhe']
lhe_files_ctau2cm/rr5.00e-08_mS2.75e+02_mT3.24e+02.lhe
24
['generate p p > Z > psiplus psiminus ', ' psiplus > psil l+ vl ', ' (psiminus\\']
['p p > Z > psiplus psiminus ', ' psiplus > psil l+ vl ', ' (psiminus > psil l- vl~ )           #Process']
['#   To display more options', ' you can type the command:               *']
['# If you want to run Pythia', ' avoid more than 50k events in a run.    *']
['# lpp: 0=No PDF', ' 1=proton', ' -1=antiproton', ' 2=photon from proton', '      *']
['     230000    = lhaid     ! if pdlabel=lhapdf', ' this is the lhapdf number']
[' 0 = ickkw            ! 0 no matching', ' 1 MLM']
['# Generation bias', ' check the wiki page below for more information:   *']
[' None = bias_module  ! Bias type of bias', ' [None', ' ptj_bias', ' -custom_folder-]']
["# Minimum and maximum pt's (for max", ' -1 means no cut)                *']
[' {}\t= pt_max_pdg ! pt cut for other particles (syntax e.g. {6: 100', 

In [7]:
def impact_with_cut(param_point, cut):
    
    folder='hepmc_output_'+param_point
    
    impact_d0(param_point)
    chooseWithCut(param_point,folder+'/pT_e.dat',folder+'/d0_e.dat','pT_e','d0_e',folder,cut)
    chooseWithCut(param_point,folder+'/pT_mu.dat',folder+'/d0_mu.dat','pT_mu','d0_mu',folder,cut)
    

In [8]:
impact_with_cut('rr1.00e-08_mS2.75e+02_mT3.24e+02',10)

['hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3053017
# events = 5422# e = 6086# mu = 3983# V(primary) = 67
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3050738
# events = 5461# e = 6258# mu = 3906# V(primary) = 38
hepmc_rr1.00e-08_mS2.75e+02_m

In [9]:
impact_with_cut('rr1.00e-08_mS2.85e+02_mT3.24e+02',10)

['hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3053020
# events = 5408# e = 5982# mu = 4030# V(primary) = 47
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3052449
# events = 5431# e = 6030# mu = 3951# V(primary) = 46
hepmc_rr1.00e-08_mS2.85e+02_m

In [10]:
impact_with_cut('rr1.00e-08_mS2.95e+02_mT3.24e+02',10)

['hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3053272
# events = 5427# e = 5942# mu = 3806# V(primary) = 45
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3054961
# events = 5388# e = 5860# mu = 3912# V(primary) = 51
hepmc_rr1.00e-08_mS2.95e+02_m

In [11]:
impact_with_cut('rr1.00e-08_mS3.04e+02_mT3.24e+02',10)

['hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3054969
# events = 5382# e = 5808# mu = 3749# V(primary) = 59
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3053505
# events = 5388# e = 5892# mu = 3667# V(primary) = 36
hepmc_rr1.00e-08_mS3.04e+02_m

In [12]:
impact_with_cut('rr1.00e-08_mS3.14e+02_mT3.24e+02',10)

['hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3057948
# events = 5203# e = 5737# mu = 3409# V(primary) = 70
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3061452
# events = 5165# e = 5626# mu = 3442# V(primary) = 52
hepmc_rr1.00e-08_mS3.14e+02_m

In [22]:
impact_with_cut('rr1.00e-08_mS1.00e+02_mT1.20e+02',10)

['hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcai']
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcaa
3032742
# events = 6153# e = 6511# mu = 4231# V(primary) = 36
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcab
3034603
# events = 6082# e = 6542# mu = 4133# V(primary) = 39
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcac
3034693
# events = 6120# e = 638

In [23]:
impact_with_cut('rr1.00e-08_mS2.00e+02_mT2.20e+02',10)

['hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcai']
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcaa
3046707
# events = 5696# e = 6133# mu = 3967# V(primary) = 34
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcab
3046263
# events = 5627# e = 6161# mu = 3801# V(primary) = 49
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcac
3047936
# events = 5601# e = 603

In [24]:
impact_with_cut('rr1.00e-08_mS4.00e+02_mT4.20e+02',10)

['hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaa
3059162
# events = 5160# e = 5691# mu = 3653# V(primary) = 60
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcab
3059507
# events = 5164# e = 5681# mu = 3630# V(primary) = 53
hepmc_rr1.00e-08_mS4.00e+02_m

In [25]:
impact_with_cut('rr1.00e-08_mS5.00e+02_mT5.20e+02',10)

['hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaa
3061234
# events = 5134# e = 5644# mu = 3552# V(primary) = 60
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcab
3061856
# events = 5078# e = 5720# mu = 3503# V(primary) = 80
hepmc_rr1.00e-08_mS5.00e+02_m

In [16]:
lifetime('rr1.00e-08_mS2.75e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3053017
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3050738
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac
3053929
hepmc_rr1.00e-08_mS2.75e+02_mT3.24e+02/tag_2_pythia8_events.h

In [17]:
lifetime('rr1.00e-08_mS2.85e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3053020
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3052449
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac
3051704
hepmc_rr1.00e-08_mS2.85e+02_mT3.24e+02/tag_2_pythia8_events.h

In [18]:
lifetime('rr1.00e-08_mS2.95e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3053272
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3054961
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac
3054141
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.h

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in sqrt
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in sqrt


hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah
3053302
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai
3051564
hepmc_rr1.00e-08_mS2.95e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj
708603


In [19]:
lifetime('rr1.00e-08_mS3.04e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3054969
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3053505
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac
3054603
hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/tag_2_pythia8_events.h

In [21]:
lifetime('rr1.00e-08_mS3.14e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcaa
3057948
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcab
3061452
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.hepmcac
3062142
hepmc_rr1.00e-08_mS3.14e+02_mT3.24e+02/tag_2_pythia8_events.h

In [26]:
lifetime('rr1.00e-08_mS1.00e+02_mT1.20e+02')

['hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcai']
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcaa
3032742
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcab
3034603
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcac
3034693
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_events.hepmcad
3036199
hepmc_rr1.00e-08_mS1.00e+02_mT1.20e+02/tag_2_pythia8_eve

In [27]:
lifetime('rr1.00e-08_mS2.00e+02_mT2.20e+02')

['hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcai']
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcaa
3046707
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcab
3046263
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcac
3047936
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_events.hepmcad
3045589
hepmc_rr1.00e-08_mS2.00e+02_mT2.20e+02/tag_2_pythia8_eve

In [28]:
lifetime('rr1.00e-08_mS4.00e+02_mT4.20e+02')

['hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcaa
3059162
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcab
3059507
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.hepmcac
3061161
hepmc_rr1.00e-08_mS4.00e+02_mT4.20e+02/tag_2_pythia8_events.h

In [29]:
lifetime('rr1.00e-08_mS5.00e+02_mT5.20e+02')

['hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaa', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcab', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcac', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcad', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcae', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaf', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcag', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcah', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcai', 'hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaj']
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcaa
3061234
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcab
3061856
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.hepmcac
3063011
hepmc_rr1.00e-08_mS5.00e+02_mT5.20e+02/tag_2_pythia8_events.h

In [27]:
# devinding lifetime in 3 peaces

var1 = np.array(genfromtxt('hepmc_output/ctau_plus.dat'))


low=var1[var1<0.1]
middle=var1[(0.1<var1)&(var1<10)]
high=var1[10<var1]


out="histos/lifetime_split"

os.makedirs(os.path.dirname(out), exist_ok=True)

with open(out+'/ctau_low.dat', 'w') as fout:
    np.savetxt(fout, low, fmt='%7f')
    
with open(out+'/ctau_middle.dat', 'w') as fout:
    np.savetxt(fout, middle, fmt='%7f')
    
with open(out+'/ctau_high.dat', 'w') as fout:
    np.savetxt(fout, high, fmt='%7f')